In [1]:
import torch
import torch.nn as nn

class GMF(nn.Module):
    def __init__(self, num_users, num_items, hidden_dim):
        super(GMF,self).__init__()
        self.gmf_user_emb = nn.Embedding(num_users, hidden_dim)
        self.gmf_item_emb = nn.Embedding(num_items, hidden_dim)
        self.gmf_linear = nn.Linear(hidden_dim, 1)
  
    def forward(self, user_indices, item_indices):
        u = self.gmf_user_emb(user_indices)
        i = self.gmf_item_emb(item_indices)
        element_product = torch.mul(u, i)
        logits = self.gmf_linear(element_product)
        result = torch.sigmoid(logits)
        return result

In [2]:
import pandas as pd
from utils.data import SampleGenerator

ratings = pd.read_csv('./dataset/ratings.csv')
ratings = ratings.rename(columns={'movieId': 'itemId'})

userId = list(set(ratings.userId))
new_userId = list(range(0,len(userId)))
df = pd.DataFrame({'userId':userId,'new_userId':new_userId})
ratings = pd.merge(ratings,df,how='left', on='userId')

itemId = list(set(ratings.itemId))
new_itemId = list(range(0,len(itemId)))
df = pd.DataFrame({'itemId':itemId,'new_itemId':new_itemId})
ratings = pd.merge(ratings,df,how='left', on='itemId')

ratings = ratings.drop(['userId', 'itemId'],axis = 1)
ratings = ratings.rename(columns={'new_userId':'userId', 'new_itemId':'itemId'})

data = SampleGenerator(ratings, implicit=True)
hidden_dim = 128
lr = 0.01
batch_size = 2048
epochs = 15

num_users = data.num_users
num_items = data.num_items
num_negatives_train = 5
num_negatives_test = 500

cuda =  torch.cuda.is_available()


model = GMF(num_users, num_items, hidden_dim)
criterion = nn.BCELoss()
optim = torch.optim.Adam(model.parameters(), lr)

if cuda:
    model.cuda()
    


C:\Users\hojin\Desktop\code\recsys-implementation-pytorch\1. Neural Collaborative Filtering\utils\data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ratings['rating'][ratings['rating'] >0] = 1.0


In [4]:
import os
from utils.eval import Evaluation

if not os.path.exists("./checkpoint"):
    os.mkdir("./checkpoint")

test_loader, negative_loader = data.instance_test_loader(num_negatives = num_negatives_test, batch_size = batch_size)
  
for epoch in range(1,epochs+1):

    train_loader = data.instance_a_train_loader(num_negatives=num_negatives_train, batch_size=batch_size)
    total_loss = 0
    for batch_id, batch in enumerate(train_loader):
        user, item, rating = batch[0], batch[1], batch[2]
        rating = rating.float()
        if cuda:
            user, item, rating = user.cuda(), item.cuda(), rating.cuda()
        optim.zero_grad()
        pred = model(user,item)
        loss = criterion(pred.view(-1), rating)
        loss.backward()
        optim.step()
        total_loss += loss.item()
    print("epoch{0} loss:{1:.4f}".format(epoch, total_loss))
  
    torch.save(model.state_dict(), "./checkpoint/gmf.pt")
  
    with torch.no_grad():
        test_users, test_items, test_preds = list(), list(), list()
        neg_users, neg_items, neg_preds = list(), list(), list()

        for batch in test_loader:
            user, item = batch[0], batch[1]
            test_users += user.data.view(-1).tolist()
            test_items += item.data.view(-1).tolist()

            if cuda:
                user, item = user.cuda(), item.cuda()
            pred = model(user,item)
            if cuda:
                pred = pred.cpu()

            test_preds += pred.data.view(-1).tolist()

        for batch in negative_loader:
            user, item = batch[0], batch[1]
            neg_users += user.data.view(-1).tolist()
            neg_items += item.data.view(-1).tolist()

            if cuda:
                user, item = user.cuda(), item.cuda()
            pred = model(user,item)
            if cuda:
                pred =  pred.cpu()

            neg_preds += pred.data.view(-1).tolist()

        eval = Evaluation([test_users, test_items, test_preds,
                           neg_users, neg_items, neg_preds])
        eval.print_eval_score_k(10)

epoch1 loss:16.8986
recall@10:0.1980, prec@10:0.0444
epoch2 loss:12.7817
recall@10:0.2008, prec@10:0.0454
epoch3 loss:8.7644
recall@10:0.2362, prec@10:0.0510
epoch4 loss:4.1618
recall@10:0.2858, prec@10:0.0644
epoch5 loss:1.0907
recall@10:0.3137, prec@10:0.0771
epoch6 loss:0.2567
recall@10:0.3236, prec@10:0.0814
epoch7 loss:0.1110
recall@10:0.3258, prec@10:0.0830
epoch8 loss:0.0663
recall@10:0.3280, prec@10:0.0852
epoch9 loss:0.0492
recall@10:0.3315, prec@10:0.0863
epoch10 loss:0.0406
recall@10:0.3366, prec@10:0.0879
epoch11 loss:0.0282
recall@10:0.3411, prec@10:0.0889
epoch12 loss:0.0203
recall@10:0.3443, prec@10:0.0900
epoch13 loss:0.0190
recall@10:0.3446, prec@10:0.0900
epoch14 loss:0.0170
recall@10:0.3498, prec@10:0.0903
epoch15 loss:0.0134
recall@10:0.3525, prec@10:0.0914
